To date, the Black-Scholes model is perhaps the most popular tool for financial derivative valuation. It is a continuous-time model for the evolution of asset prices. Here we will discuss the
main assumptions to the model, treat simulation techniques and derive the famous Black-Scholes
formula for European options.
Although it is more realistic than the binomial tree, the Black-Scholes model is still a simplified
representation of the market. Let again S denote the value of a stock. The main concept here is
that the risk-neutral dynamics of the stock price are captured by a geometric Brownian motion.
This is a stochastic process that is described by the following SDE.


dSt = rStdt + σStdzt




In [1]:
import numpy as np
import math